In [1]:
import platform
import os
if platform.system() == 'Darwin':
    DATA_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Data.nosync"
    ROOT_PATH = "/Users/maltegenschow/Documents/Uni/Thesis/Thesis"
elif platform.system() == 'Linux':
    DATA_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync"
    ROOT_PATH = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Thesis"

current_wd = os.getcwd()

In [2]:
import numpy as np
from interfacegan_utils import calculate_boundary
from glob import glob
import pandas as pd
from tqdm import tqdm

In [3]:
latent_input_base_path = f"{DATA_PATH}/Models/InterfaceGAN/Inputs/e4e_00005/"
all_paths = glob(f"{latent_input_base_path}/*/*")
all_paths = [path for path in all_paths if not "summary_stats" in path]

In [4]:
for path in tqdm(all_paths):
    out_dir = path.replace('Inputs', 'Outputs')
    if os.path.exists(out_dir):
        continue

    else:
        accuracies = []
        os.makedirs(out_dir, exist_ok=True)
        for dim in range(16):
            latent_codes_path = f"{path}/latents_dim_{dim}.npy"
            scores_path = f"{path}/target.npy"
            out_name = f'boundary_dim{dim}.npy'
            acc = calculate_boundary(out_dir, out_name, latent_codes_path, scores_path)
            accuracies.append(acc)

        # save result
        attribute = path.split('/')[-2]
        values = path.split('/')[-1].split('_')

        # Crate dataframe with columns 'attribute', 'value1, 'value2', 'accuracy1', 'accuracy2', ...
        result = pd.DataFrame(columns=['attribute'] + ['neg_class', 'pos_class'] + [f'acc_dim_{i}' for i in range(len(accuracies))])
        result.loc[0] = [attribute] + values + accuracies
        result.to_csv(f"{out_dir}/summary_stats.csv", index=False)

100%|██████████| 174/174 [00:00<00:00, 126264.51it/s]


In [5]:
summary_paths = glob(f"{latent_input_base_path.replace('Inputs', 'Outputs')}/*/*/summary_stats.csv")

training_results = pd.concat([pd.read_csv(path) for path in summary_paths])
training_results

,attribute,neg_class,pos_class,acc_dim_0,acc_dim_1,acc_dim_2,acc_dim_3,acc_dim_4,acc_dim_5,acc_dim_6,acc_dim_7,acc_dim_8,acc_dim_9,acc_dim_10,acc_dim_11,acc_dim_12,acc_dim_13,acc_dim_14,acc_dim_15
0,sleeve_length,Sleeveless,Elbow length,0.993028,0.995020,0.995020,0.996016,0.997012,0.990040,0.998008,0.996016,0.997012,0.992032,0.995020,0.996016,0.992032,0.994024,0.996016,0.996016
0,sleeve_length,Extra short,Elbow length,0.984055,0.986333,0.984055,0.995444,0.993166,0.984055,0.997722,0.984055,0.981777,0.990888,0.988610,0.981777,0.990888,0.995444,0.993166,0.984055
0,sleeve_length,34 length,Mixed,0.957831,0.969880,0.977912,0.959839,0.961847,0.971888,0.963855,0.949799,0.969880,0.945783,0.959839,0.963855,0.963855,0.975904,0.971888,0.967871
0,sleeve_length,Mixed,Extra long,0.941441,0.959459,0.954955,0.936937,0.954955,0.941441,0.959459,0.923423,0.905405,0.914414,0.918919,0.900901,0.932432,0.945946,0.932432,0.936937
0,sleeve_length,Extra short,Strapless,0.944649,0.952030,0.963100,0.974170,0.970480,0.970480,0.985240,0.974170,0.944649,0.959410,0.963100,0.937269,0.966790,0.944649,0.948339,0.959410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,color,orange,purple,0.891525,0.911864,0.908475,0.894915,0.888136,0.898305,0.877966,0.905085,0.861017,0.901695,0.891525,0.922034,0.908475,0.901695,0.942373,0.969492
0,color,black,yellow,0.997196,0.994393,0.994393,0.992523,0.997196,0.994393,0.996262,0.992523,0.994393,0.992523,0.995327,0.996262,0.996262,0.995327,0.996262,0.998131
0,color,brown,grey,0.879464,0.870536,0.863095,0.845238,0.840774,0.843750,0.857143,0.852679,0.869048,0.849702,0.864583,0.870536,0.885417,0.858631,0.885417,0.909226
0,color,blue,white,0.978454,0.979721,0.977186,0.973384,0.978454,0.980989,0.977186,0.980989,0.980989,0.982256,0.972117,0.968314,0.983523,0.979721,0.977186,0.984791
